In [1]:
# This notebook is  used to evaluate perofrmance of our RAG ingestion and query pipeline.
# Method:
#   - Generated test data using a sample GitHub repository (offline process not in this notebook)
#     - https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/
#   - Use our ingestion pipeline to parse the same reporisiroty and index in our vector store
#   - Use RAGAS on teadt dataset + our answers fromn our RAG and visualise the mterics.
import os
import sys
import pandas as pd

sys.path.insert(1, '/home/jovyan/work/code')
from config import VectorDBConfig, EmbeddingConfig, ProcessingConfig, ChatConfig
from config_helper import ConfigHelper
from pipeline import DocumentPipeline
from localrag import LocalRAG

from TraceSetup import get_tracer, get_logger
logger = get_logger()
tracer = get_tracer()

config_helper = ConfigHelper(False)

In [2]:
# Ingest a GitHub repository and import into our vector store

# Initialize the pipeline
pipeline = DocumentPipeline(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config,
    logger=logger,
    tracer=tracer)

local_ingestion_file="dotnet-docs-aspire.txt"
repository="https://github.com/dotnet/docs-aspire"
with tracer.start_as_current_span(f"Starting ingesting {repository} using local ingestion file {local_ingestion_file}"):
    pipeline.process_single_file(local_ingestion_file,repository)


/home/jovyan/work/code/pipeline.py:38: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)
INFO:TraceSetup:collection: llamaball-minilmm vector size: 384
INFO:TraceSetup:Collection llamaball-minilmm does not exist. Will create now. Vector size is 384
INFO:TraceSetup:Created collection: llamaball-minilmm
INFO:TraceSetup:Starting repository chunking...
INFO:TraceSetup:Created chunking strategies
INFO:TraceSetup:Split content into 181 files
INFO:TraceSetup:Total chunks created: 3342


Total parts found: 363
Total files processed: 181
Sample file paths:
 1. README.md
 2. CODE_OF_CONDUCT.md
 3. SECURITY.md
Added 50 chunks for file docs/reference/aspire-faq.yml


INFO:TraceSetup:done processing the repository.


done processing the repository.


In [3]:
# Test our RAG Solution

def demonstrate_local_rag(rag):
    """Demonstrate how to use the LocalRAG class."""    
    # Example questions to test
    questions = [
        "If I use Docker Compose, why should I care about .Net Aspire?",
        "Is .Net Aspire an alternative to Kubernetes?"
    ]
    with tracer.start_as_current_span("Entering questions loop."):
        for question in questions:
            print(f"Question: {question}")
            # print("\nRelevant chunks:")
            # with tracer.start_as_current_span("rag get chunks"):
            #     chunks = rag.get_relevant_chunks(question, k=5)
            #     for i, chunk in enumerate(chunks, 1):
            #         print(chunk)
            #         break
#                    print(f"\nChunk {i}:")
#                    print(f"Source: {chunk.metadata.get('file_path', 'Unknown')}")
#                    print(f"Content: {chunk.page_content[:200]}...")
                    
            print("\nGenerated Answer:")
            with tracer.start_as_current_span("Retrieve answers."):
                answer = rag.retrieve_and_answer(question, k=10)
                print(answer)
                print("\n" + "="*80 + "\n")

rag = LocalRAG(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config, 
    chat_config=config_helper.chat_config
)
with tracer.start_as_current_span("Starting demo"):
    demonstrate_local_rag(rag)

Question: If I use Docker Compose, why should I care about .Net Aspire?

Generated Answer:


/home/jovyan/work/code/localrag.py:42: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)


If you are using Docker Compose to orchestrate your projects and executables, here's why you might want to consider using .NET Aspire instead:

1. **Easier app composition**: While Docker Compose is great for managing individual containers, it can make it more difficult to specify the exact components of your application. .NET Aspire provides a unified way to compose applications across multiple services, making it easier to manage dependencies and orchestrate complex systems.

2. **Improved service discovery**: .NET Aspire allows you to inject connection strings, network configurations, and other necessary information for your services into your host application, simplifying the process of managing connections between services. This is particularly useful when working with cloud-based applications that require configuration files or environment variables.

3. **Better support for multi-language development**: While Docker Compose has some limitations in terms of language support (it o